# Setup the metric
we need a metric "function" class.  
It should only do the logic needed,   
in our case, we want to measure how often the label and the prediction are the same

In [ ]:
from TorchMetricLogger import TorchMetricLogger as TML
from TorchMetricLogger import TmlMetric, TmlLoss, TmlMetricFunction
import torch

class TMLBinaryAccuracy(TmlMetricFunction):
    def __init__(self):
        super().__init__()
        
    def calculate(self, gold_labels, predictions):
        return (gold_labels > 0.5) == (predictions > 0.5)

binary_accuracy = TMLBinaryAccuracy()

We now simulate one epoch with 3 batches.  
We can also set weights, to e.g. ignore some samples or make them less important.  
Here we set every missclassification with a weight of zero to basically ignore it.

In [ ]:
labels = torch.ones(100)
predictions = torch.zeros(100)
weights = torch.zeros(100)

tml = TML()

tml(partial=True, 
    train_bin_accuracy=TmlMetric(labels, predictions, binary_accuracy, weights=weights)
)

predictions = torch.ones(100)
weights = torch.ones(100)

tml(partial=True, 
    train_bin_accuracy=TmlMetric(labels, predictions, binary_accuracy, weights=weights)
)

half_positive = torch.zeros(100)
half_positive[50:] = 1
tml(partial=True, 
    train_bin_accuracy=TmlMetric(labels, half_positive, binary_accuracy, weights=half_positive)
)

tml.batch_end()

assert tml.history["train_bin_accuracy"] == [1] 